In [2]:
import json
import time
import datetime

# path to credentials.json
credentials_path = "../config/credentials.json"

# load API key
with open(credentials_path, 'r') as file:
    credentials = json.load(file)
    api_key = credentials.get("riot_api_key")

In [ ]:
import requests
import pandas as pd
import os

# API headers
headers = {
    'X-Riot-Token': api_key
}

# base API endpoint
riot_base_url = "https://<region>.api.riotgames.com/lol"

## Main Purpose of Notebook is to Test API requests for Scripts

In [31]:
'''
Functions to grab champion, rune and item data
'''

def fetch_champion_data_ddragon():
    """Fetch champion data from DDragon."""
    url = "https://ddragon.leagueoflegends.com/cdn/14.23.1/data/en_US/champion.json"  
    response = requests.get(url)
    
    if response.status_code == 200:
        champions = response.json()
        return champions
    else:
        print(f"Error fetching champion data: {response.status_code}")
        return None

def fetch_item_data_ddragon():
    """Fetch item data from DDragon."""
    url = "http://ddragon.leagueoflegends.com/cdn/14.23.1/data/en_US/item.json" 
    response = requests.get(url)
    
    if response.status_code == 200:
        champions = response.json()
        return champions
    else:
        print(f"Error fetching champion data: {response.status_code}")
        return None

def fetch_rune_data_ddragon():
    """Fetch rune data from DDragon."""
    url = "https://ddragon.leagueoflegends.com/cdn/14.23.1/data/en_US/runesReforged.json"  # Replace with the latest patch
    response = requests.get(url)
    
    if response.status_code == 200:
        runes = response.json()
        return runes
    else:
        print(f"Error fetching rune data: {response.status_code}")
        return None
        

In [36]:
# fetch champion data using DDragon
champion_data = fetch_champion_data_ddragon()

# fetch item data using DDragon
item_data = fetch_item_data_ddragon()

# fetch rune data using DDragon
rune_data = fetch_rune_data_ddragon()

# save to JSON file
if champion_data:
    output_path = "../data/raw/champion_data/champions.json"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as file:
        json.dump(champion_data, file)
    print(f"Champion data saved to {output_path}")

if item_data:
    output_path = "../data/raw/item_data/items.json"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as file:
        json.dump(item_data, file)
    print(f"Champion data saved to {output_path}")

if rune_data:
    output_path = "../data/raw/runes_data/runes.json"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as file:
        json.dump(rune_data, file)
    print(f"Champion data saved to {output_path}")

Champion data saved to ../data/raw/champion_data/champions.json
Champion data saved to ../data/raw/item_data/items.json
Champion data saved to ../data/raw/runes_data/runes.json


In [71]:
import os
import json
import time
import requests
import datetime

def fetch_summoner_ids_by_multiple_ranks(api_key, region, output_path):
    """
    Fetch summoner IDs by multiple ranks and divisions, and append to a JSON file.

    Parameters:
        api_key (str): Riot API key.
        region (str): Riot region for league data (e.g., "na1").
        output_path (str): Path to save the output JSON file.

    Returns:
        dict: Updated dictionary of summoner IDs categorized by rank.
    """
    # define the tiers and divisions to iterate over
    tiers = {
        "DIAMOND": ["IV", "III", "II", "I"]
    }

    high_tiers = ["MASTER", "GRANDMASTER", "CHALLENGER"]

    # initialize dictionary to hold all summoner IDs
    all_summoner_ids = {"DIAMOND": [], "MASTER": [], "GRANDMASTER": [], "CHALLENGER": []}

    # initialize rate limit tracking
    request_count = 0
    start_time = datetime.datetime.now()

    try:
        # load existing data if file exists, ensuring correct format
        if os.path.exists(output_path):
            with open(output_path, 'r') as f:
                existing_summoner_ids = json.load(f)
                if not isinstance(existing_summoner_ids, dict):
                    # reformat existing data to dictionary if it's a list or incorrect format
                    existing_summoner_ids = {"DIAMOND": [], "MASTER": [], "GRANDMASTER": [], "CHALLENGER": []}
        else:
            existing_summoner_ids = {"DIAMOND": [], "MASTER": [], "GRANDMASTER": [], "CHALLENGER": []}

        # fetch diamond summoner IDs
        for tier, divisions in tiers.items():
            for division in divisions:
                page = 1
                while True:
                    # check rate limit
                    current_time = datetime.datetime.now()
                    elapsed_seconds = (current_time - start_time).total_seconds()

                    # if 2 minutes have passed, reset the request count and start time
                    if elapsed_seconds > 120:
                        request_count = 0
                        start_time = current_time

                    # if we're close to the 100 requests per 2-minute limit, wait
                    if request_count >= 90:
                        sleep_time = 120 - elapsed_seconds
                        print(f"Approaching 2-minute rate limit. Waiting for {sleep_time:.2f} seconds...")
                        time.sleep(sleep_time)
                        request_count = 0
                        start_time = datetime.datetime.now()

                    # build the url for Diamond tier
                    url = f"https://{region}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}"
                    params = {"page": page}
                    headers = {"X-Riot-Token": api_key}

                    try:
                        # make request
                        response = requests.get(url, headers=headers, params=params)
                        request_count += 1

                        # check rate limit
                        if response.status_code == 429:
                            retry_after = int(response.headers.get("Retry-After", 10))  # Use Retry-After if provided, otherwise 10 seconds
                            print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
                            time.sleep(retry_after)
                            continue

                        # raise http errors
                        response.raise_for_status()

                        # parse the response and extract summoner IDs
                        summoner_entries = response.json()

                        # stop if no more data is available on this page
                        if not summoner_entries:
                            break

                        # extract summoner IDs
                        new_summoner_ids = [entry["summonerId"] for entry in summoner_entries]
                        all_summoner_ids["DIAMOND"].extend(new_summoner_ids)
                        page += 1  

                        # adding delay
                        if request_count % 20 == 0:
                            print("20 requests made in the last second. Pausing for 1 second to avoid limit...")
                            time.sleep(1)

                    except requests.exceptions.RequestException as e:
                        print(f"Request error for {tier} {division}, page {page}: {e}")
                        break

        # fetch High Tier Summoner IDs (Master, Grandmaster, Challenger)
        for high_tier in high_tiers:
            # specific endpoint for each high tier
            url = f"https://{region}.api.riotgames.com/lol/league/v4/{high_tier.lower()}leagues/by-queue/RANKED_SOLO_5x5"
            headers = {"X-Riot-Token": api_key}

            try:
                # make the request
                response = requests.get(url, headers=headers)
                request_count += 1

                # check for rate limit
                if response.status_code == 429:
                    retry_after = int(response.headers.get("Retry-After", 10))  # Use Retry-After if provided, otherwise 10 seconds
                    print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
                    time.sleep(retry_after)
                    continue

                # raise http errors
                response.raise_for_status()

                # parse the response and extract summoner IDs
                summoner_entries = response.json()["entries"]
                new_summoner_ids = [entry["summonerId"] for entry in summoner_entries]
                all_summoner_ids[high_tier].extend(new_summoner_ids)

                # adding delay
                if request_count % 20 == 0:
                    print("20 requests made in the last second. Pausing for 1 second to avoid limit...")
                    time.sleep(1)

            except requests.exceptions.RequestException as e:
                print(f"Request error for {high_tier}: {e}")
                continue

        # merge and remove duplicates for each rank
        for rank in all_summoner_ids:
            all_summoner_ids[rank] = list(set(existing_summoner_ids.get(rank, []) + all_summoner_ids[rank]))

        # ensure output directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # write updated data to file
        with open(output_path, 'w') as f:
            json.dump(all_summoner_ids, f, indent=4)

        print(f"Summoner IDs updated and saved to {output_path}")
        return all_summoner_ids

    except Exception as e:
        print(f"Unexpected error: {e}")
        return {}

# Example usage
API_KEY = api_key
REGION = "na1"
OUTPUT_PATH = "../data/raw/summoner_id/summoner_id.json"

summoner_ids = fetch_summoner_ids_by_multiple_ranks(
    api_key=API_KEY, 
    region=REGION, 
    output_path=OUTPUT_PATH
)

20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
Approaching 2-minute rate limit. Waiting for 82.10 seconds...
20 requests made in the last second. Pausing for 1 second to avoid limit...
Summoner IDs updated and saved to ../data/raw/summoner_id/summoner_id.json


In [85]:
def fetch_puuids(api_key, summoner_ids_path, region, output_path):
    """
    Fetch PUUIDs for summoners given their summoner IDs.

    Parameters:
    - api_key: str, your Riot Games API key.
    - summoner_ids_path: str, path to the JSON file containing summoner IDs.
    - region: str, region for summoner data (e.g., "na1").
    - output_path: str, path to save the output JSON file for PUUIDs.

    Returns:
    - puuids: list, list of PUUIDs for all summoners.
    """
    # load summoner IDs from the given JSON file
    if not os.path.exists(summoner_ids_path):
        print(f"Summoner IDs file not found: {summoner_ids_path}")
        return []

    with open(summoner_ids_path, 'r') as f:
        summoner_data = json.load(f)

    # combine all summoner IDs from different branches into a single list
    summoner_ids = []
    for rank in summoner_data:
        summoner_ids.extend(summoner_data[rank])

    puuids = []
    headers = {"X-Riot-Token": api_key}

    # load existing PUUIDs if the output file exists
    if os.path.exists(output_path):
        with open(output_path, 'r') as f:
            existing_puuids = json.load(f)
    else:
        existing_puuids = []

    # initialize rate limit tracking
    request_count = 0
    start_time = datetime.datetime.now()

    for summoner_id in summoner_ids:
        # check rate limits
        current_time = datetime.datetime.now()
        elapsed_seconds = (current_time - start_time).total_seconds()

        # reset the request count after 2 minutes
        if elapsed_seconds > 120:
            request_count = 0
            start_time = current_time

        # if approaching the 2-minute limit, wait
        if request_count >= 90:
            sleep_time = 120 - elapsed_seconds
            print(f"Approaching 2-minute rate limit. Waiting for {sleep_time:.2f} seconds...")
            time.sleep(sleep_time)
            request_count = 0
            start_time = datetime.datetime.now()

        # fetch PUUID using summonerId
        try:
            url = f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
            response = requests.get(url, headers=headers)
            request_count += 1

            # rate limit per second (20 requests per second)
            if request_count % 20 == 0:
                print("20 requests made in the last second. Pausing for 1 second to avoid limit...")
                time.sleep(1)

            response.raise_for_status()
            summoner_data = response.json()
            puuid = summoner_data["puuid"]
            if puuid not in existing_puuids and puuid not in puuids:
                puuids.append(puuid)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching PUUID for summonerId {summoner_id}: {e}")
            continue

    # combine new PUUIDs with existing ones and remove duplicates
    all_puuids = list(set(existing_puuids + puuids))

    # ensure the output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # save updated PUUIDs to JSON file
    with open(output_path, 'w') as f:
        json.dump(all_puuids, f, indent=4)

    print(f"PUUIDs updated and saved to {output_path}")
    return all_puuids

# example Usage
API_KEY = api_key
SUMMONER_IDS_PATH = "../data/raw/summoner_id/summoner_id.json"
REGION = "na1"
OUTPUT_PATH = "../data/raw/puuids/puuids.json"

puuids = fetch_puuids(
    api_key=API_KEY, 
    summoner_ids_path=SUMMONER_IDS_PATH, 
    region=REGION, 
    output_path=OUTPUT_PATH
)

20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
Approaching 2-minute rate limit. Waiting for 92.47 seconds...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
Approaching 2-minute rate limit. Waiting for 92.41 seconds...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pau

KeyboardInterrupt: 

In [13]:
'''

Fetching Match Ids

'''

def fetch_match_ids(api_key, puuid, region, count=10):
    """
    Fetch match IDs for a given PUUID, filtering for ranked solo/duo games only.

    Parameters:
    - api_key: str, Riot Games API key.
    - puuid: str, PUUID of the summoner.
    - region: str, region for match data (e.g., "americas").
    - count: int, number of matches to retrieve.

    Returns:
    - match_ids: list of match IDs.
    """
    headers = {"X-Riot-Token": api_key}
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&start=0&count={count}"
    # queue=420 ensures only ranked solo/duo matches are fetched
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        match_ids = response.json()
        return match_ids
    except requests.exceptions.RequestException as e:
        print(f"Error fetching match IDs for PUUID {puuid}: {e}")
        return []

def load_puuids(puuids_path):
    """
    Load PUUIDs from a JSON file.

    Parameters:
    - puuids_path: str, path to the JSON file containing PUUIDs.

    Returns:
    - list of PUUIDs.
    """
    if not os.path.exists(puuids_path):
        print(f"PUUIDs file not found: {puuids_path}")
        return []

    with open(puuids_path, 'r') as f:
        puuids_data = json.load(f)
        return list(puuids_data.values())[::-1] if isinstance(puuids_data, dict) else puuids_data[::-1]

# example usage
API_KEY = load_api_key()
PUUIDS_PATH = "../data/raw/puuids/puuids.json"
REGION = "americas"
MATCH_OUTPUT_PATH = "../data/raw/match_ids/match_ids.json"

# load PUUIDs from file
puuids = load_puuids(PUUIDS_PATH)

# fetch match IDs for the last PUUID and save to JSON
if puuids:
    match_ids = fetch_match_ids(API_KEY, puuids[0], REGION)

    # ensure the output directory exists
    os.makedirs(os.path.dirname(MATCH_OUTPUT_PATH), exist_ok=True)

    # load existing match IDs to handle duplicates
    if os.path.exists(MATCH_OUTPUT_PATH):
        with open(MATCH_OUTPUT_PATH, 'r') as f:
            existing_match_ids = set(json.load(f))
    else:
        existing_match_ids = set()

    # add new match IDs, avoiding duplicates
    unique_match_ids = list(existing_match_ids.union(match_ids))

    # save match IDs to JSON file
    with open(MATCH_OUTPUT_PATH, 'w') as f:
        json.dump(unique_match_ids, f, indent=4)

    print(f"Fetched match IDs saved to {MATCH_OUTPUT_PATH}")
else:
    print("No PUUIDs found in the provided file.")


Fetched match IDs saved to ../data/raw/match_ids/match_ids.json


In [26]:
'''
Sample of match details
'''

import os
import json
import requests

def load_api_key():
    """
    Load the API key from the credentials file.
    """
    with open("../config/credentials.json", 'r') as f:
        data = json.load(f)
    return data.get("riot_api_key")

def fetch_raw_match_data(api_key, match_id, region):
    """
    Fetch the raw match details as a JSON string.

    Parameters:
    - api_key: str, your Riot Games API key.
    - match_id: str, match ID.
    - region: str, region for match data (e.g., "americas").

    Returns:
    - match_data: dict, raw JSON data about the match.
    """
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"HTTP error occurred: {e}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching match data: {e}")

# example usage
API_KEY = load_api_key()
REGION = "americas"  
MATCH_ID = "NA1_5164719703"

match_data = fetch_raw_match_data(API_KEY, MATCH_ID, REGION)
if match_data:
    print(json.dumps(match_data, indent=4))


{
    "metadata": {
        "dataVersion": "2",
        "matchId": "NA1_5164719703",
        "participants": [
            "RF2OTZQ_Sx7QHzfdK1mngGiHostoJvMmrwMja3oqln0XgT8T-V2fslZRNjQ3ATUgMQO_dVQd9pQfSw",
            "DqRcXqzVHetvSu9FTp23ib91IaDSDifX5PrJVnJOyHnUzGnOHq2iVk2F5oOwqEQibC9XYdv8WYdFIQ",
            "agpoZKOv-unJB1mCwYcEZZkqJuqPLAqvPnJL4_Xs4ar8sj-uz-g7dShxRDY7D8m1QxbdjgJ9O52wWA",
            "5e8wDa1-3IxAf9DFjspM-pdDO-FIuJw4A8C-FlO2ARR8T0eTLPyMPaIfIXY-h-HTucvedXM2rFuR2Q",
            "6LhLWIANPHQOXg8dEO_ggwMJdVb7mN23-huMi6Oe3FP42000gRDS6pUzOdXdmREA7GEcI6UxCRyJaw",
            "WaflSlN8JJrEWvXCPm_zeYC8XrSBmT2VeaU0Je03dNtWDfkcqSj86CCh1FVkrTFHo-k-BvnMcTkt4A",
            "0jkbkXXIBQ_w7G4PP4B3o7w47uMFJRzfY1eHdnA-Eo8ULF8qMEqab2K3tGwRwhTCLShgefp2Jw5iEg",
            "62kQxER6sDrs3Xu1-3Ib1nec_NESe7iL6MWP2yTKrqDEnpx9pDxa_HMevXh7x1HVOGTfRwZFZjxp4g",
            "0OYGo-5qyrwQ2DKDTuSoZgYvbBLYTVlYxPsafJQ01lBGURyjOP6PRrEluceVC3Fzl4dsEetzVoFRvA",
            "t1vPILhKHOTy8w-C09q6mhMxfzO_TV

In [61]:
'''
POTENTIAL SCRIPT FOR MATCH DETAILS
'''

import os
import json
import requests
import pandas as pd

def load_api_key():
    """
    Load the API key from the credentials file.
    """
    with open("../config/credentials.json", 'r') as f:
        data = json.load(f)
    return data.get("riot_api_key")

def fetch_match_details(api_key, match_id, region):
    """
    Fetch raw match data from Riot API.
    """
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def extract_match_details(match_data):
    """
    Extract relevant match details from the match data.
    """
    participants = match_data['info']['participants']
    extracted_data = []

    for participant in participants:
        # find the corresponding matchup champion
        matchup_champion = next((p['championName'] for p in participants 
                                 if p['individualPosition'] == participant['individualPosition'] and 
                                 p['teamId'] != participant['teamId']), None)

        match_summary = {
            "matchId": match_data["metadata"]["matchId"],
            "gameDuration": match_data["info"]["gameDuration"],
            "championId": participant["championId"],
            "championName": participant["championName"],
            "teamId": participant["teamId"],
            "individualPosition": participant.get("individualPosition", "Unknown"),
            "kills": participant["kills"],
            "deaths": participant["deaths"],
            "assists": participant["assists"],
            "win": participant["win"],
            "goldEarned": participant["goldEarned"],
            "totalDamageDealt": participant["totalDamageDealtToChampions"],
            "totalDamageTaken": participant["totalDamageTaken"],
            "totalHeal": participant["totalHeal"],
            "matchupChampion": matchup_champion
        }

        # add items and their purchase times as individual columns
        for i in range(6):
            item_key = f"item{i}"
            match_summary[f"item_{i}"] = participant.get(item_key, "Unknown Item")
            match_summary[f"item_purchase_time_{i}"] = participant.get('itemPurchaseTime', {}).get(item_key, "Unknown Time")

        extracted_data.append(match_summary)

    return extracted_data

# example usage
API_KEY = load_api_key()
REGION = "americas"
MATCH_IDS_PATH = "../data/raw/match_ids/match_ids.json"
MATCH_DETAILS_OUTPUT_PATH = "../data/raw/match_details/"

# ensure the output directory exists
os.makedirs(MATCH_DETAILS_OUTPUT_PATH, exist_ok=True)

# load match IDs
if os.path.exists(MATCH_IDS_PATH):
    with open(MATCH_IDS_PATH, 'r') as f:
        match_ids = json.load(f)
else:
    match_ids = []

# initialize a list to store all extracted data
all_match_details = []

# process each match ID and extract details
for match_id in match_ids:
    try:
        # fetch match details
        match_data = fetch_match_details(API_KEY, match_id, REGION)

        # extract relevant details
        extracted_details = extract_match_details(match_data)

        # add extracted details to the list
        all_match_details.extend(extracted_details)

        print(f"Details for match {match_id} processed.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching details for match ID {match_id}: {e}")

# convert all match details into a DataFrame
match_details_df = pd.DataFrame(all_match_details)

# save the DataFrame as a CSV file
match_details_df.to_csv(os.path.join(MATCH_DETAILS_OUTPUT_PATH, "all_match_details.csv"), index=False)

print("All match details have been processed and saved to CSV.")



Details for match NA1_5159373690 processed.
Details for match NA1_5159534194 processed.
Details for match NA1_5154055544 processed.
Details for match NA1_5155002995 processed.
Details for match NA1_5136762466 processed.
Details for match NA1_5145724360 processed.
Details for match NA1_5146571389 processed.
Details for match NA1_5153510550 processed.
Details for match NA1_5158857489 processed.
Details for match NA1_5154645037 processed.
Details for match NA1_5145136202 processed.
Details for match NA1_5157407198 processed.
Details for match NA1_5147085054 processed.
Details for match NA1_5156578876 processed.
Details for match NA1_5154210614 processed.
Details for match NA1_5154253391 processed.
Details for match NA1_5153117852 processed.
Details for match NA1_5161639810 processed.


KeyboardInterrupt: 

In [78]:
import os
import json
import requests

def load_api_key():
    """
    Load the API key from the credentials file.
    """
    with open("../config/credentials.json", 'r') as f:
        data = json.load(f)
    return data.get("riot_api_key")

def fetch_match_details(api_key, match_id, region):
    """
    Fetch raw match data from Riot API.
    """
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def fetch_match_timeline(api_key, match_id, region):
    """
    Fetch match timeline data from Riot API.
    """
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline"
    headers = {"X-Riot-Token": api_key}

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()

def extract_match_details(match_data, timeline_data):
    """
    Extract relevant match details from the match data and match timeline.
    """
    participants = match_data['info']['participants']
    extracted_data = []

    for participant in participants:
        # find the corresponding matchup champion
        matchup_champion = next((p['championName'] for p in participants 
                                 if p['individualPosition'] == participant['individualPosition'] and 
                                 p['teamId'] != participant['teamId']), None)

        # initialize item purchase times with "Unknown Time"
        item_purchase_times = {f"item_purchase_time_{i}": "Unknown Time" for i in range(6)}

        # extract item purchase times from the timeline
        participant_id = participant["participantId"]
        for frame in timeline_data["info"]["frames"]:
            for event in frame["events"]:
                if (
                    event["type"] == "ITEM_PURCHASED" and
                    event["participantId"] == participant_id
                ):
                    item_id = event["itemId"]
                    timestamp = event["timestamp"] // 1000  # convert milliseconds to seconds

                    # find the appropriate item slot
                    for i in range(6):
                        if participant.get(f"item{i}") == item_id:
                            item_purchase_times[f"item_purchase_time_{i}"] = timestamp
                            break

        # extract rune information
        perks = participant.get("perks", {})
        primary_perk = perks.get("styles", [])[0] if perks.get("styles") else {}
        secondary_perk = perks.get("styles", [])[1] if len(perks.get("styles", [])) > 1 else {}

        match_summary = {
            "matchId": match_data["metadata"]["matchId"],
            "gameDuration": match_data["info"]["gameDuration"],
            "championId": participant["championId"],
            "championName": participant["championName"],
            "teamId": participant["teamId"],
            "individualPosition": participant.get("individualPosition", "Unknown"),
            "kills": participant["kills"],
            "deaths": participant["deaths"],
            "assists": participant["assists"],
            "win": participant["win"],
            "goldEarned": participant["goldEarned"],
            "totalDamageDealt": participant["totalDamageDealtToChampions"],
            "totalDamageTaken": participant["totalDamageTaken"],
            "totalHeal": participant["totalHeal"],
            "matchupChampion": matchup_champion,
            "primaryRune": primary_perk.get("style", "Unknown"),
            "primaryRunes": [slot.get("perk", "Unknown") for slot in primary_perk.get("selections", [])],
            "secondaryRune": secondary_perk.get("style", "Unknown"),
            "secondaryRunes": [slot.get("perk", "Unknown") for slot in secondary_perk.get("selections", [])]
        }

        # add items and their purchase times as individual columns
        for i in range(6):
            item_key = f"item{i}"
            match_summary[f"item_{i}"] = participant.get(item_key, "Unknown Item")
            match_summary[f"item_purchase_time_{i}"] = item_purchase_times[f"item_purchase_time_{i}"]

        extracted_data.append(match_summary)

    return extracted_data

# example usage
API_KEY = load_api_key()
REGION = "americas"
MATCH_IDS_PATH = "../data/raw/match_ids/match_ids.json"
MATCH_DETAILS_OUTPUT_PATH = "../data/raw/match_details/all_match_details.json"

# ensure the output directory exists
os.makedirs(os.path.dirname(MATCH_DETAILS_OUTPUT_PATH), exist_ok=True)

# load match IDs
if os.path.exists(MATCH_IDS_PATH):
    with open(MATCH_IDS_PATH, 'r') as f:
        match_ids = json.load(f)
else:
    match_ids = []

# load existing match details if the output file already exists
if os.path.exists(MATCH_DETAILS_OUTPUT_PATH):
    with open(MATCH_DETAILS_OUTPUT_PATH, 'r') as f:
        all_match_details = json.load(f)
else:
    all_match_details = []

# check if we have match IDs available
if match_ids:
    first_match_id = match_ids[0]  # using only the first match ID
    try:
        # fetch match details and timeline
        match_data = fetch_match_details(API_KEY, first_match_id, REGION)
        timeline_data = fetch_match_timeline(API_KEY, first_match_id, REGION)

        # extract relevant details
        extracted_details = extract_match_details(match_data, timeline_data)

        # append new match details to the existing data
        all_match_details.extend(extracted_details)

        # save updated match details to JSON file
        with open(MATCH_DETAILS_OUTPUT_PATH, 'w') as f:
            json.dump(all_match_details, f, indent=4)

        print(f"Details for match {first_match_id} processed and saved to JSON.")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching details for match ID {first_match_id}: {e}")
else:
    print("No match IDs found in the provided file.")


Details for match NA1_5159373690 processed and saved to JSON.
